In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
data=pd.read_csv('./final_dataset.csv')

In [33]:
#for col in data.columns:
    #print(col)
#print(data[' Protocol'])
data=data[(data[' Source Port']==22) | (data[' Destination Port']==22) ]
print(len(data))

9073


In [62]:
%matplotlib inline
from mpl_toolkits.mplot3d import Axes3D

from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

cols=[' Protocol',' Flow Duration',' Total Fwd Packets',' Total Backward Packets','Total Length of Fwd Packets',' Total Length of Bwd Packets',' Fwd Packet Length Max',' Fwd Packet Length Min',' Fwd Packet Length Mean',' Fwd Packet Length Std','Bwd Packet Length Max',' Bwd Packet Length Min',' Bwd Packet Length Mean',' Bwd Packet Length Std','Flow Bytes/s',' Flow Packets/s',' Flow IAT Mean',' Flow IAT Std',' Flow IAT Max',' Flow IAT Min','Fwd IAT Total',' Fwd IAT Mean',' Fwd IAT Std',' Fwd IAT Max',' Fwd IAT Min','Bwd IAT Total',' Bwd IAT Mean',' Bwd IAT Std',' Bwd IAT Max',' Bwd IAT Min','Fwd PSH Flags',' Bwd PSH Flags',' Fwd URG Flags',' Bwd URG Flags',' Fwd Header Length',' Bwd Header Length','Fwd Packets/s',' Bwd Packets/s',' Min Packet Length',' Max Packet Length',' Packet Length Mean',' Packet Length Std',' Packet Length Variance','FIN Flag Count',' SYN Flag Count',' RST Flag Count',' PSH Flag Count',' ACK Flag Count',' URG Flag Count',' CWE Flag Count',' ECE Flag Count',' Down/Up Ratio',' Average Packet Size',' Avg Fwd Segment Size',' Avg Bwd Segment Size',' Fwd Header Length','Fwd Avg Bytes/Bulk',' Fwd Avg Packets/Bulk',' Fwd Avg Bulk Rate',' Bwd Avg Bytes/Bulk',' Bwd Avg Packets/Bulk','Bwd Avg Bulk Rate','Subflow Fwd Packets',' Subflow Fwd Bytes',' Subflow Bwd Packets',' Subflow Bwd Bytes','Init_Win_bytes_forward',' Init_Win_bytes_backward',' act_data_pkt_fwd',' min_seg_size_forward','Active Mean',' Active Std',' Active Max',' Active Min','Idle Mean',' Idle Std',' Idle Max',' Idle Min']

dfa_old = data[cols]    
dfa=dfa_old[np.isfinite(dfa_old).all(1)]

print('Total Row : ', len(dfa))
## Feature Scaling
sc_dfa = StandardScaler()
np.isnan(dfa)
dfa_std = sc_dfa.fit_transform(dfa.astype(np.float64))
## Clustering with KMeans
kmeans = KMeans(n_clusters=2, random_state=42).fit(dfa_std)
labels = kmeans.labels_
new_dfa = pd.DataFrame(data = dfa_std, columns = cols)
new_dfa['label_kmeans'] = labels

count=[0,0,0,0,0]
for i in new_dfa['label_kmeans']:
    count[i]+=1



Total Row :  9069


/media/saikiran/Common disk/RnD1/env/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [63]:
print(count[0])
print(count[1])
print(count[2])
print(count[3])
print(count[4])

5113
3956
0
0
0


In [64]:
count1=[0,0]
for i in data[' Label']:
    if i=="BENIGN":
        count1[0]+=1
    else:
        count1[1]+=1


In [65]:
print(count1[0],count1[1])

3176 5897


In [66]:

acc_benign=0
bad_benign=0
acc_attack=0
bad_attack=0
total=0

for i in range(len(new_dfa['label_kmeans'])):
    if new_dfa['label_kmeans'].iloc[i] ==1 and data[' Label'].iloc[i]=='BENIGN':
        acc_benign+=1
        total+=1
    elif data[' Label'].iloc[i]=='BENIGN' and new_dfa['label_kmeans'].iloc[i]!=1:
        bad_benign+=1
        total+=1
    elif data[' Label'].iloc[i]!='BENIGN' and new_dfa['label_kmeans'].iloc[i]==1:
        bad_attack+=1
        total+=1
    elif data[' Label'].iloc[i]!='BENIGN' and new_dfa['label_kmeans'].iloc[i]!=1:
        acc_attack+=1
        total+=1


In [68]:
print(acc_benign,bad_benign,acc_attack,bad_attack)
if acc_attack+acc_benign+bad_benign+bad_attack==total:
    print(total)

1411 1764 3349 2545
9069


Lets apply supervised machine learning techniques, cause the performance of unsupervised machine learning algorithms are very poor.

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import sklearn

cols=[' Protocol',' Flow Duration',' Total Fwd Packets',' Total Backward Packets','Total Length of Fwd Packets',' Total Length of Bwd Packets',' Fwd Packet Length Max',' Fwd Packet Length Min',' Fwd Packet Length Mean',' Fwd Packet Length Std','Bwd Packet Length Max',' Bwd Packet Length Min',' Bwd Packet Length Mean',' Bwd Packet Length Std','Flow Bytes/s',' Flow Packets/s',' Flow IAT Mean',' Flow IAT Std',' Flow IAT Max',' Flow IAT Min','Fwd IAT Total',' Fwd IAT Mean',' Fwd IAT Std',' Fwd IAT Max',' Fwd IAT Min','Bwd IAT Total',' Bwd IAT Mean',' Bwd IAT Std',' Bwd IAT Max',' Bwd IAT Min','Fwd PSH Flags',' Bwd PSH Flags',' Fwd URG Flags',' Bwd URG Flags',' Fwd Header Length',' Bwd Header Length','Fwd Packets/s',' Bwd Packets/s',' Min Packet Length',' Max Packet Length',' Packet Length Mean',' Packet Length Std',' Packet Length Variance','FIN Flag Count',' SYN Flag Count',' RST Flag Count',' PSH Flag Count',' ACK Flag Count',' URG Flag Count',' CWE Flag Count',' ECE Flag Count',' Down/Up Ratio',' Average Packet Size',' Avg Fwd Segment Size',' Avg Bwd Segment Size',' Fwd Header Length','Fwd Avg Bytes/Bulk',' Fwd Avg Packets/Bulk',' Fwd Avg Bulk Rate',' Bwd Avg Bytes/Bulk',' Bwd Avg Packets/Bulk','Bwd Avg Bulk Rate','Subflow Fwd Packets',' Subflow Fwd Bytes',' Subflow Bwd Packets',' Subflow Bwd Bytes','Init_Win_bytes_forward',' Init_Win_bytes_backward',' act_data_pkt_fwd',' min_seg_size_forward','Active Mean',' Active Std',' Active Max',' Active Min','Idle Mean',' Idle Std',' Idle Max',' Idle Min',' Label']


data=pd.read_csv('./final_dataset.csv')
print(len(data.columns))
data=data[(data[' Source Port']==22) | (data[' Destination Port']==22)]

data[' Label'][data[' Label']=='SSH-Patator']=1
data[' Label'][data[' Label']=='BENIGN']=0


data=data[cols]
data=data.astype(np.float64)
data=data[np.isfinite(data).all(1)]

y=data[' Label']
y=y.to_numpy()

data.drop(data.columns[len(data.columns)-1], axis=1, inplace=True)
x=data.to_numpy()


85


/tmp/ipykernel_146843/1705385293.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[' Label'][data[' Label']=='BENIGN']=0


In [67]:
print(x.shape)
print(y.shape)

(9069, 78)
(9069,)


In [68]:
x=x-np.mean(x,axis=0)

In [72]:
from sklearn.model_selection import train_test_split

x=np.reshape(x,(x.shape[0],13,6,1))
x=x.astype(np.float64)
x=x*255
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2)

In [73]:
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(7255, 13, 6, 1)
(1814, 13, 6, 1)
(7255,)
(1814,)


In [74]:
print(np.sum(y_train))
print(len(y_train))

4731.0
7255


Dataset contains 4724 flows which are Brute force labeled and rest are benign, Which implies dataset it evenly balanced.

In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPool2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense



ModuleNotFoundError: No module named 'tensorflow'

In [4]:
import pandas as pd

data=pd.read_csv('./final_dataset.csv')
for i in range(len(data.columns)):
    print(i+1,"&",data.columns[i],"\\\\",)

1 & Flow ID \\
2 &  Source IP \\
3 &  Source Port \\
4 &  Destination IP \\
5 &  Destination Port \\
6 &  Protocol \\
7 &  Timestamp \\
8 &  Flow Duration \\
9 &  Total Fwd Packets \\
10 &  Total Backward Packets \\
11 & Total Length of Fwd Packets \\
12 &  Total Length of Bwd Packets \\
13 &  Fwd Packet Length Max \\
14 &  Fwd Packet Length Min \\
15 &  Fwd Packet Length Mean \\
16 &  Fwd Packet Length Std \\
17 & Bwd Packet Length Max \\
18 &  Bwd Packet Length Min \\
19 &  Bwd Packet Length Mean \\
20 &  Bwd Packet Length Std \\
21 & Flow Bytes/s \\
22 &  Flow Packets/s \\
23 &  Flow IAT Mean \\
24 &  Flow IAT Std \\
25 &  Flow IAT Max \\
26 &  Flow IAT Min \\
27 & Fwd IAT Total \\
28 &  Fwd IAT Mean \\
29 &  Fwd IAT Std \\
30 &  Fwd IAT Max \\
31 &  Fwd IAT Min \\
32 & Bwd IAT Total \\
33 &  Bwd IAT Mean \\
34 &  Bwd IAT Std \\
35 &  Bwd IAT Max \\
36 &  Bwd IAT Min \\
37 & Fwd PSH Flags \\
38 &  Bwd PSH Flags \\
39 &  Fwd URG Flags \\
40 &  Bwd URG Flags \\
41 &  Fwd Header Length